In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data = pd.read_csv('data/BO_2w_smooth.csv')

Q_11 = data['Q11']
Q_22 = data['Q22']

alphas_read = data['alpha']

In [ ]:
from numba import njit
from tqdm import tqdm
from scipy.linalg import sqrtm

@njit
def softmax(x, beta=1):
    max_x = np.array([np.max(row) for row in x])

    x = x - max_x.reshape(-1,1)
    P = np.exp(beta*x)    
    return (P.T / np.sum(P,1)).T


@njit
def A(z1):
    return softmax(np.outer(z1,z1)) + np.eye(len(z1))


@njit
def g(z1, z2):
    A1 = A(z1) @ z2
    return softmax(np.outer(A1, A1))


samples = 100000
gen_error_list = np.zeros(len(Q_11))

for i in tqdm(range(len(Q_11))):
    Q = np.array([[Q_11[i], 0], [0, Q_22[i]]])
    gen_error = 0
    for _ in range(samples):

        Z = np.random.normal(0,1, (2,2))
        U = np.random.normal(0,1, (2,2))

        sqrt_Q = sqrtm(Q)
        sqrt_one_minus_Q = sqrtm(np.eye(Q.shape[0]) - Q)
        omega = sqrt_Q @ Z

        Z_true = sqrt_Q@Z + sqrt_one_minus_Q@U
        y_teacher = g(Z_true[0], Z_true[1])
        y_student = g(omega[0], omega[1])

        loss = np.linalg.norm(y_teacher - y_student)**2
        gen_error += loss

    gen_error_list[i] = gen_error / samples

    print(f"alpha = {alphas_read[i]}: {gen_error_list[i]}")


  1%|          | 1/181 [00:27<1:22:22, 27.46s/it]

alpha = 0.0: 0.3848642790595911


  1%|          | 2/181 [00:50<1:14:57, 25.13s/it]

alpha = 0.1435844654940499: 0.3863464685630178


  2%|▏         | 3/181 [01:19<1:18:53, 26.59s/it]

...

alpha = 0.3923884510511811: 0.2987083693504203


 13%|█▎        | 24/181 [08:55<55:38, 21.26s/it]

alpha = 0.4036745402787401: 0.2910213203526811


 14%|█▍        | 25/181 [09:20<58:28, 22.49s/it]

alpha = 0.4149606295062992: 0.27786065341361693


In [8]:
[print(f"{alphas_read[i]},{gen_error_list[i]}") for i in range(len(alphas_read))]

1

0.0,0.38790104430716293
0.0333333333,0.3863906533072164
0.0666666666,0.3873364436237081
0.0999999999,0.386496188506728
0.1333333332,0.387630446654175
0.1666666665,0.3853618503105733
0.1999999998,0.37943140255793223
0.2333333331,0.3714551620391603
0.2666666664,0.36006952922175506
0.2999999997,0.34930170951738093
0.333333333,0.3367345044354671
0.3666666663,0.31950025703925283
0.3999999996,0.29281596862582837
0.4333333329,0.26521979324409845
0.4666666662,0.2290801256030667
0.4999999995,0.18290987814475676
0.5333333328,0.14144536786699358
0.5666666661,0.10963932437731191
0.5999999994,0.0936929839179832
0.6333333327,0.07715788296286483
0.666666666,0.06711189035625276
0.6999999993,0.06360388166891703
0.7333333326,0.05551124290970098
0.7666666659,0.05330829745085989
0.7999999992,0.04656697225578734
0.8333333325,0.0356888556706998
0.8666666658,0.0340239109828602
0.8999999991,0.029007817411543086
0.9333333324,0.025145068887988502
0.9666666657,0.021862992937828875
0.999999999,0.02222946822097011

1

In [9]:
from numba import njit
from tqdm import tqdm
from scipy.linalg import sqrtm

@njit
def softmax(x, beta=1):
    max_x = np.array([np.max(row) for row in x])

    x = x - max_x.reshape(-1,1)
    P = np.exp(beta*x)    
    return (P.T / np.sum(P,1)).T


@njit
def A(z1):
    return softmax(np.outer(z1,z1)) + np.eye(len(z1))


@njit
def g(z1, z2):
    A1 = A(z1) @ z2
    return softmax(np.outer(A1, A1))


samples = 100000
gen_error_list = np.zeros(len(Q_11))

for i in tqdm(range(len(Q_11))):
    Q = np.array([[0, 0], [0, Q_22[i]]])
    gen_error = 0
    for _ in range(samples):

        Z = np.random.normal(0,1, (2,2))
        U = np.random.normal(0,1, (2,2))

        sqrt_Q = sqrtm(Q)
        sqrt_one_minus_Q = sqrtm(np.eye(Q.shape[0]) - Q)
        omega = sqrt_Q @ Z

        Z_true = sqrt_Q@Z + sqrt_one_minus_Q@U
        y_teacher = g(Z_true[0], Z_true[1])
        y_student = g(omega[0], omega[1])

        loss = np.linalg.norm(y_teacher - y_student)**2
        gen_error += loss

    gen_error_list[i] = gen_error / samples

    print(f"alpha = {alphas_read[i]}: {gen_error_list[i]}")


  2%|▏         | 1/48 [00:25<19:40, 25.12s/it]

alpha = 0.0: 0.38576924742253743


  4%|▍         | 2/48 [00:45<16:54, 22.06s/it]

alpha = 0.0333333333: 0.38983397654849256


  6%|▋         | 3/48 [01:10<17:38, 23.51s/it]

...

alpha = 1.5333333318: 0.03564259218913689


 98%|█████████▊| 47/48 [17:04<00:21, 21.42s/it]

alpha = 1.5666666651: 0.0351472086849386


100%|██████████| 48/48 [17:25<00:00, 21.79s/it]

alpha = 1.5999999984: 0.03496864335677616


In [10]:
[print(f"{alphas_read[i]},{gen_error_list[i]}") for i in range(len(alphas_read))]

1

0.0,0.38576924742253743
0.0333333333,0.38983397654849256
0.0666666666,0.3850014758635755
0.0999999999,0.385858019658417
0.1333333332,0.38620451975527176
0.1666666665,0.3847197827928925
0.1999999998,0.3801219578078816
0.2333333331,0.3699448507361799
0.2666666664,0.3605752863131721
0.2999999997,0.3489031507809933
0.333333333,0.33666168477253305
0.3666666663,0.3210228414262696
0.3999999996,0.2910472694203277
0.4333333329,0.26272098324974125
0.4666666662,0.2299044128310558
0.4999999995,0.18411141652858354
0.5333333328,0.14222092977268705
0.5666666661,0.10946260442095497
0.5999999994,0.09405243524869136
0.6333333327,0.07827819193503689
0.666666666,0.06684159133233106
0.6999999993,0.06331542854006347
0.7333333326,0.05512545651053185
0.7666666659,0.05268983636418027
0.7999999992,0.04667273740044811
0.8333333325,0.041005449980894865
0.8666666658,0.042359338227994085
0.8999999991,0.04016062642240004
0.9333333324,0.03973070191805872
0.9666666657,0.03802049324635839
0.999999999,0.0382071055436034

1